## Walk through of a sample PCL code:


### Lesson 1 Create and SVM with iSCSI Example

[create_san.py](../../create_san.py)

#### The Header Comment Section

This section usually includes details such as 
- the file name, 
- author, 
- date, 
- purpose of the program, and any other relevant information. 

It’s a good practice to include such comments for better readability and maintainability of the code 

In [ ]:
#! /usr/local/bin/python3
"""
Purpose: Script to create sn SVM by using the netapp_ontap library.
         It will create a Group, a lun and map the igroup to the lun.
Author: Vish Hulikal
Usage: python3.11 igroup.py [-h] -c CLUSTER -vs VSERVER_NAME, -l LUN_NAME -ig IGROUP_NAME
       [-u API_USER] [-p API_PASS]
"""

#### The Input Section:

Since we are using a Jupyter Notebook to run the code and not running the code from the command line, we need this section to simulate entering the arguments from the command line.

When you run the notebook, for example by clicking on the `Run All` button, a dialog prompt will appear at the top of the window. You will then need to enter the command line arguments as described in the usage section above.

If you do not provide the user, `admin`is used. If you do not provide the password, you will be prompted for the password.

Suggested command arguments:

In [ ]:
import sys

# Prompt the user to enter command line arguments
args = input("Please enter command line arguments: ")

# Split the entered string into a list of arguments
args = args.split()

# Assign the list of arguments to sys.argv
sys.argv = ['ipykernel_launcher.py'] + args

#### Import Section

Here we will import the following modules:
  - [`argparse`](https://pypi.org/project/argparse/) : This is a popular python module. The argparse module makes it easy to write user friendly command line interfaces. 
  The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.
  - [`getpass`](https://docs.python.org/3/library/getpass.html) : Used to Prompt the user for a password without echoing
  - [`logging`](https://pypi.org/project/logging/) : This module is intended to provide a standard error logging mechanism in Python as per PEP 282.
  - [`netapp_ontap.config`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/config.html) : This module contains the global configuration options and related functions for the library.
  - [`netapp_ontap.host_connection`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/host_connection.html) : This module defines a host connection object which is used to communicate with the API host
  - [`netapp_ontap.error`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/error.html) : This module defines the custom exception type. All exceptions raised by the library descend from this type
  - [`netapp_ontap.resources.svm`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/svm.html) : Managing SVMs
  - [`netapp_ontap.resources.lun`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/lun.html) : Manage LUNs 
  - [`netapp_ontap.resources.igroup`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/igroup.html) : Manage iGroups
  - [`netapp_ontap.resources.lun_map`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/lun_map.html) : Manage Mapping LUNs


In [ ]:

import argparse
from getpass import getpass
import logging

from netapp_ontap import config, HostConnection, NetAppRestError
from netapp_ontap.resources import Lun, Igroup, LunMap

#### Function Definitions

These functions are defined:
  
  - `create_igroup`
  - `create_lun`  
  - `create_lun_map`  
  
  

In [ ]:
def create_igroup(igroup_name: str, vserver_name: str) -> None:
    """Create an Igroup on the SVM"""

    data = {
        'name': igroup_name,
        'svm': {'name': vserver_name},
        'protocol': 'iscsi',
        'os_type': 'windows'
    }

    igroup = Igroup(**data)

    try:
        igroup.post()
        print("Igroup %s created successfully" % igroup.name)
    except NetAppRestError as err:
        print("Error: Igroup was not created: %s" % err)
    return

In [ ]:

def create_lun(lun_name: str, vserver_name: str, lun_size: int) -> None:
    """Creates a new lun in a volume"""

    data = {
        'name': lun_name,
        'svm': {'name': vserver_name},
        'space': {'size': lun_size},
        'os_type': 'windows'
    }

    lun = Lun(**data)

    try:
        lun.post()
        print("Lun %s created successfully" % lun.name)
    except NetAppRestError as err:
        print("Error: Lun was not created: %s" % err)
    return


In [ ]:

def create_lun_map(vserver_name: str, igroup_name: str, lun_name: str) -> None:
    """Create a map between Lun and the igroup"""

    data = {
        'svm': {'name': vserver_name},
        'igroup': {'name': igroup_name},
        'lun': {'name': lun_name}
    }

    lun_map = LunMap(**data)

    try:
        lun_map.post()
        print("Lun Map %s created successfully" % lun_map.igroup.name)
    except NetAppRestError as err:
        print("Error: Lun Map was not created: %s" % err)
    return


#### Arguments Parser

We define which arguments need to be passed to the script and argparse does the rest...

In [ ]:


def parse_args() -> argparse.Namespace:
    """Parse the command line arguments from the user"""

    parser = argparse.ArgumentParser(
        description="This script will create a SAN configuration"
    )
    parser.add_argument(
        "-c", "--cluster", required=True, help="Cluster Name"
   )
    parser.add_argument(
        "-vs", "--vserver_name", required=True, help="VServer name"
    )
    parser.add_argument(
        "-l", "--lun_name", required=True, help="LUN path-name"
    )
    parser.add_argument(
        "-ig", "--igroup_name", required=True, help="Igroup name"
    )

    parser.add_argument("-u", "--api_user", default="admin", help="API Username")
    parser.add_argument("-p", "--api_pass", help="API Password")
    parsed_args = parser.parse_args()

    # collect the password without echo if not already provided
    if not parsed_args.api_pass:
        parsed_args.api_pass = getpass()

    return parsed_args


#### The Main Section

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
       format="[%(asctime)s] [%(levelname)5s] [%(module)s:%(lineno)s] %(message)s",
    )

    args = parse_args()
    config.CONNECTION = HostConnection(
        args.cluster, username=args.api_user, password=args.api_pass, verify=False,
    )

    create_igroup(args.igroup_name, args.vserver_name)
    create_lun(args.lun_name, args.vserver_name, 30000000)
    create_lun_map(args.vserver_name, args.igroup_name, args.lun_name)


---

---

[Back to Module 6 Demo - Notebook:](./Module6_demo.ipynb)